In [1]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
import pprint
import os
from bs4 import BeautifulSoup
import requests
import json
import jsonschema
from jsonschema import validate
import re
from datetime import datetime

In [3]:
# Get a client (the "core" client provides access to projects, teams, etc)
def get_clients_projects():

    # Fill in with your personal access token and org URL
    personal_access_token = os.getenv("gitpat")
    organization_url = 'https://dev.azure.com/supportedcustomers'

    # Create a connection to the org
    credentials = BasicAuthentication('', personal_access_token)
    connection = Connection(base_url=organization_url, creds=credentials)    
    
    core_client = connection.clients.get_core_client()
    git_client = connection.clients.get_git_client()

    # Get the first page of projects
    get_projects_response = core_client.get_projects()
    return core_client, git_client, get_projects_response

In [4]:
def json_validator(data):
    try:
        json.loads(data)
        return True
    except ValueError as error:
        print("invalid json: %s" % error)
        return False

In [5]:
def recurse_git_tree(repo_id, object_id=None):
    print(f'step one')
    tree = git_client.get_tree(repository_id=repo.id, sha1=object_id)

    for entry in tree.tree_entries:
        if entry.relative_path=="pipeline":
            print(f'step two')
            recurse_git_tree(repo_id=repo_id, object_id=entry.object_id)
        elif entry.git_object_type =='blob':
            print(f'step three')
            blob = git_client.get_blob_content(repository_id=repo.id, sha1=entry.object_id)
            s = json.dumps(json.loads(next(blob)))

            # [Copy, Dataflow]

            try:
                if s.index("\"type\": \"Copy\"")>0:               
                    print(f'before: {s}')
                    found_value = s[s.index("\"timeout\""): s.find(",", s.index("timeout"))]
                    s = s.replace(found_value,'"timeout": "5:00:00"')
                    found_value = s[s.index("\"retryIntervalInSeconds\""): s.find(",", s.index("retryIntervalInSeconds"))]
                    s = s.replace(found_value,'"retryIntervalInSeconds": "2200"')
                    found_value = s[s.index("\"retry\""): s.find(",", s.index("retry"))]
                    s = s.replace(found_value,'"retry": "1"')
                    print(f'after: {s}')
            except Exception as e:
                print(e)
                continue
        else:
            recurse_git_tree(repo_id=repo_id, object_id=entry.object_id)


### Work in progress

In [ ]:
core_client, git_client, get_projects_response = get_clients_projects()

index = 0
while get_projects_response is not None:
    print('did i get')
    for project in get_projects_response.value:
        repos = git_client.get_repositories(project.id)
        for repo in repos:
            if repo.name=="noname":
                try:
                    branches = git_client.get_branches(repository_id=repo.id)
                    for branch in branches:
                        
                        print(f'The repository {repo.name} has the following branch {branch.name}')
                        commit = git_client.get_commit(repository_id=repo.id, commit_id=branch.commit.commit_id)

                        recurse_git_tree(repo_id=repo.id, object_id=commit.tree_id)

                except Exception as e:
                    print(e)
                    continue
            index += 1
    if get_projects_response.continuation_token is not None and get_projects_response.continuation_token != "":
        # Get the next page of projects
        get_projects_response = core_client.get_projects(continuation_token=get_projects_response.continuation_token)

    else:
        # All projects have been retrieved
        get_projects_response = None

The repository noname has the following branch adf2_test1
{
    'additional_properties': {}, 
    'git_object_type': 'tree', 
    'mode': '40000', 
    'object_id': 'aa4e7709a0d2577a07e31358f06dac9657d00963', 
    'relative_path': 'pipeline', 
    'size': 89, 
    'url': 'https://dev.azure.com/SupportedCustomers/e6091a89-c16c-440b-a71b-e4d9151542a1/_apis/git/repositories/6b058b4d-221d-4ca1-905b-5f2eda2a2462/trees/aa4e7709a0d2577a07e31358f06dac9657d00963'
}

In [7]:
core_client, git_client, get_projects_response = get_clients_projects()

from base64 import b64encode
from base64 import decode

branch_to_find = 'test3a'

retryIntervalInSeconds = 50
retry = 1
timeout = "0:05:00"




index = 0
while get_projects_response is not None:
    for project in get_projects_response.value:
        repos = git_client.get_repositories(project.id)
        for repo in repos:
            try:
                branches = git_client.get_branches(repository_id=repo.id)
                for branch in branches:
                    if branch.name != branch_to_find:
                        print(f'These are not the droids you are looking for: {branch.name}')
                        continue


                    print(f'Am I in test3a: {"test3a" in branch.name}')
                    commit = git_client.get_commit(repository_id=repo.id, commit_id=branch.commit.commit_id)
                    tree = git_client.get_tree(repository_id=repo.id, sha1=commit.tree_id)

                    for entry in tree.tree_entries:
                        if entry.relative_path == "pipeline":
                            cc = git_client.get_tree(repository_id=repo.id, sha1=entry.object_id)
                            for c in cc.tree_entries:
                                if c.git_object_type == 'blob':
                                    blob = git_client.get_blob_content(repository_id=repo.id, sha1=c.object_id)
                                    try:
                                        data = json.loads(next(blob))
                                    except Exception as ex:
                                        print(f'failed to read json file {ex}')
                                    s = json.dumps(data)
                                    print(f'what did it look like before {s}')

                                    if s.index("\"type\": \"Copy\""):               

                                        print(f'branch name is {branch.name}')
                                        found_value = s[s.index("\"timeout\""): s.find(",", s.index("timeout"))]
                                        s = s.replace(found_value,f'"timeout": {timeout}') # 0.0:00:00
                                        found_value = s[s.index("\"retryIntervalInSeconds\""): s.find(",", s.index("retryIntervalInSeconds"))]
                                        s = s.replace(found_value,f'"retryIntervalInSeconds": {retryIntervalInSeconds}')
                                        found_value = s[s.index("\"retry\""): s.find(",", s.index("retry"))]
                                        s = s.replace(found_value,f'"retry": {retry}')

                                        try:
                                            s = json.loads(s)
                                        except Exception as mm:
                                            print(f'error on converting to dict: {mm}')
                                        
                                        # this is close but AzDO doesn't decode this and leaves it as bytes
                                        # my_json = b64encode(json.dumps(s,indent=4).encode('ascii'))
                                        # string_byte_json = str(my_json).replace("'b","").replace("'","")

                                        my_json =   json.dumps(s,indent=4)

                                        push = {"refUpdates":[{"name":"refs/heads/" + branch.name,"oldObjectId": commit.commit_id }],"commits":[{"comment":"Updating readme via python and Azure DevOps SDK: " + datetime.now().strftime("%Y %B, %A %w, %H hours %M minutes")  ,"changes":[{"changeType":"edit","item":{"path":"/pipeline/" + c.relative_path},"newContent":{"content": my_json ,"contentType":"rawText"}}]}]}

                                        git_client.create_push(
                                            repository_id=repo.id, push=push
                                        )

            except Exception as e:
                print(e)
        index += 1

        #project.
    if get_projects_response.continuation_token is not None and get_projects_response.continuation_token != "":
        # Get the next page of projects
        get_projects_response = core_client.get_projects(continuation_token=get_projects_response.continuation_token)

    else:
        # All projects have been retrieved
        get_projects_response = None

These are not the droids you are looking for: Ecolab
These are not the droids you are looking for: Ecolab2a
These are not the droids you are looking for: Ecolab2b
These are not the droids you are looking for: adf2_test1
These are not the droids you are looking for: adf2_test1a
These are not the droids you are looking for: f12f1401-revert-from-Ecolab
These are not the droids you are looking for: main
These are not the droids you are looking for: test1
These are not the droids you are looking for: test2
These are not the droids you are looking for: test4a
These are not the droids you are looking for: main
These are not the droids you are looking for: adf_publish
These are not the droids you are looking for: main
These are not the droids you are looking for: test1
These are not the droids you are looking for: test2
Am I in test3a: True
what did it look like before {"name": "pipeline1", "properties": {"activities": [{"name": "CopyDataDemo", "type": "Copy", "dependsOn": [], "policy": {"time